# Imports

In [ ]:
import ctypes
import matplotlib.pyplot as plt
import numpy as np
from Utils.library_overcoat import *

# Load library
rml = ctypes.CDLL("../../Library/target/debug/librust_ml.dylib")

# Cas de tests

## Régression Linéaire

### Linear simple 2D

In [ ]:
X = np.array([
      [1],
      [2]
], dtype=np.float64)
Y = np.array([
      2,
      3
], dtype=np.float64)

plt.scatter(X, Y, color='red')

#### conversion des valeurs pour correspondre convention C

In [ ]:
cx = X.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
cy = Y.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
c_len_x = ctypes.c_int(len(X))

#### Imports du modèle

In [ ]:
# Init functions
rml.create_linear_regression_model.restype = ctypes.c_void_p
rml.train_linear_regression_model.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_double),
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_size_t
]
rml.delete_linear_regression_model.argtypes = [
    ctypes.c_void_p
]

#### création du modèle

In [ ]:
model = rml.create_linear_regression_model()

#### Entrainement du modèle

In [ ]:
rml.train_linear_regression_model(ctypes.byref(model), cx, cy, c_len_x)

#### Prédiction du modèle

In [ ]:
x = 2
c_input = ctypes.c_double(x)
predicted = rml.predict_linear_regression_model(model, c_input)

l = np.linspace(0, 8, 7)
f = model.coefficient * l + model.constant
plt.plot(l, f)
plt.grid(True)
plt.show()

#### Suppression du modèle

In [ ]:
rml.delete_linear_regression_model(model)

## Classification PMC

### Linear Simple

#### Imports modèle

In [ ]:
rml.delete_mlp_model.argtypes = [ctypes.c_void_p]
rml.create_pmc_model.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int]
rml.create_pmc_model.restype = ctypes.c_void_p

rml.predict_pmc.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_double),
                            ctypes.c_int, ctypes.c_bool]
rml.predict_pmc.restype = ctypes.POINTER(ctypes.c_float)
rml.train_pmc_model.argtypes = [ctypes.c_void_p,
                                ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.c_int,
                                ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.c_int,
                                ctypes.c_double, ctypes.c_int, ctypes.c_bool]
rml.export_pmc.argtypes = [ctypes.c_void_p]
rml.export_pmc.restype = ctypes.c_char_p
rml.load_pmc_json.argtypes = [ctypes.c_char_p, ctypes.c_void_p]

### Création du modèle

In [ ]:
param = np.array([2, 1], dtype=np.int32)
c_param = param.ctypes.data_as(ctypes.POINTER(ctypes.c_int))

mlp_ptr = rml.create_pmc_model(c_param, ctypes.c_int(len(param)))

#### Entrainement du Modèle

In [ ]:
i_test = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0]], dtype=np.float32)
arr = i_test.flatten()
o_test = np.array([[1.0], [-1.0], [-1.0]], dtype=np.float32)
arr2 = o_test.flatten()

# Conversion
i_ptr = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
c_length = ctypes.c_int(len(arr))
c_sub_length = ctypes.c_int(len(i_test[0]))

o_ptr = arr2.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
o_length = ctypes.c_int(len(arr2))
o_sub_length = ctypes.c_int(len(o_test[0]))

alpha = ctypes.c_double(0.01)
iteration = ctypes.c_int(100000)
is_classification = ctypes.c_bool(True)

# Entrainement
rml.train_pmc_model(mlp_ptr, i_ptr, c_length, c_sub_length, o_ptr, o_length, o_sub_length, alpha, iteration,
                    is_classification)

#### Prédiction

In [ ]:
if mlp_ptr is not None:
    test_1_all_samples_inputs = np.array([
        [0.0, 0.0],
        [0.0, 1.0],
        [1.0, 0.0]
    ], dtype=np.float32)
    test_1_all_samples_expected_outputs = np.array([
        [1.0],
        [-1.0],
        [-1.0]
    ], dtype=np.float32)

    res = []
    for sample_inputs in test_1_all_samples_inputs:
        sample_ptr = sample_inputs.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
        sample_length = ctypes.c_int(len(sample_inputs))

        res_ptr = rml.predict_pmc(mlp_ptr, sample_ptr, sample_length, ctypes.c_bool(True))
        res_array = np.zeros(len(sample_inputs), dtype=np.float32)
        for i in range(len(sample_inputs)):
            res_array[i] = res_ptr[i]
        res.append([res_array[0]])

    print("--------------------- Predict ---------------------")
    print(res)

#### Export du modèle

In [ ]:
pmc_data_c_string = rml.export_pmc(mlp_ptr)
pmc_data_str = ctypes.cast(pmc_data_c_string, ctypes.c_char_p).value.decode("utf-8")

pmc_data_json = json.loads(pmc_data_str)

with open("notebook_pmc_data.json", "w") as file:
    json.dump(pmc_data_json, file)

#### Import du modèle

In [ ]:
with open("pmc_data.json", "r") as file:
    json_data = file.read()

rml.load_pmc_json(json_data.encode("utf-8"), mlp_ptr)